In [1]:
#%xmode Verbose

In [2]:
import sys
sys.path.append("../libs/")
sys.path.append("../../../deep-belief-network/")

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import shift_join_data, print_hp, flatten
from keras_tuner import HyperModel, Objective
import datetime
import keras_tuner as kt
import os
from dbn.models import SupervisedDBNRegression
from keras_tuner.oracles import RandomSearchOracle
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit

# Carga de datos

In [4]:
df = pd.read_csv('../data/wb_dataset_prep.csv')
df = df.drop('country',axis=1)
iso = df['iso'].unique()    #Codigos de paises
df = df.set_index(['iso','year'])

# Dividir Datos

In [5]:
iso_test = ['PER']
iso_train = iso[(iso != iso_test[0])]
target_col = ['rgdp_growth']
features = df.columns[(df.columns!=target_col[0])]

# Normalizar

In [6]:
df_x_train = df.loc[iso_train][features].copy()
df_y_train = df.loc[iso_train][target_col].copy()

scaler_x_train = StandardScaler()
scaler_y_train = StandardScaler()

df_x_train.iloc[:,:] = scaler_x_train.fit_transform(df_x_train)
df_y_train.iloc[:,:] = scaler_y_train.fit_transform(df_y_train)

df_x_train.iloc[:,:] = np.clip(df_x_train,-5,5)
df_y_train.iloc[:,:] = np.clip(df_y_train,-5,5)

# Caracteristicas pasadas y futuras

In [7]:
n_steps_in = 10
n_steps_out = 3
x_s,y_s = shift_join_data(df_x_train, df_y_train, iso_train, n_steps_in, n_steps_out)
x_s = flatten(x_s)

# Optimizacion

In [8]:
class DBNHyperModel(HyperModel):

    def build(self, hp):
        print("Pasa aqui 1")
        # Parametrizamos nro de capas, nro de nodos y ratio de aprendizaje
        hp_rbm_layers = hp.Int('rbm_layers',1,3,step=1)
        hp_rbm_nodes = hp.Int('rbm_nodes',96,356,step=16)
        h_layers_structure = [hp_rbm_nodes for i in range(hp_rbm_layers)]
        hp_rbm_learning_rate = hp.Choice('rbm_learning_rate', values=[1e-3, 1e-4, 1e-5])
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
        hp_activation = hp.Choice('activation', values=['relu','tanh'])
        hp_n_epochs = hp.Int('rbm_epochs',10,30,step=10)
        n_iter_backprop = 100
        mini_batch = 32

        return SupervisedDBNRegression(
            hidden_layers_structure = h_layers_structure,
            learning_rate_rbm = hp_rbm_learning_rate,
            learning_rate = hp_learning_rate,
            n_epochs_rbm = hp_n_epochs,
            n_iter_backprop = n_iter_backprop,
            batch_size = mini_batch,
            activation_function = hp_activation
        )

In [11]:
n_steps_out = 3
n_features = len(features)
ajuste_path = os.path.normpath('G:/')
fecha_hora = datetime.datetime.now().strftime('%Y%m%d_%H%M')
objective = Objective('score','min')
max_trials = 4

In [12]:
name_prj = 'DBN'+'_'+fecha_hora
dbn_hypermodel = DBNHyperModel()

dbn_tuner = kt.SklearnTuner(
    hypermodel = dbn_hypermodel,
    oracle= RandomSearchOracle(
        objective = objective,
        max_trials = max_trials
    ),
    scoring = metrics.make_scorer(metrics.mean_squared_error),
    cv = TimeSeriesSplit(n_splits=5),
    directory = ajuste_path,
    project_name = name_prj,
    overwrite=True)

dbn_tuner.search(x_s, y_s)


Trial 4 Complete [00h 14m 05s]
score: 0.4424203869486682

Best score So Far: 0.4424203869486682
Total elapsed time: 00h 29m 57s
INFO:tensorflow:Oracle triggered exit


In [13]:
# guardar mejor modelo
output_path = 'ajustes/'
best_dbn_hps = dbn_tuner.get_best_hyperparameters(num_trials = 1)[0]
print_hp(output_path+name_prj+'.txt',dbn_tuner)

dbn_model = dbn_tuner.hypermodel.build(best_dbn_hps)
dbn_model.save(output_path+name_prj+'.h5')

Pasa aqui 1


In [ ]:
# #sys.path.append("../libs/")
# from notifications import enviar_correo
# enviar_correo("Ajuste de Parametros Finalizado!","Se ha completado: {}".format(name_prj))